In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import time
import pandas as pd
import os

Model Definition

In [ ]:
inputSize = 517
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(inputSize, 2048)
        self.hidden1 = nn.Linear(2048, 1024)
        self.hidden2 = nn.Linear(1024, 512)
        self.hidden3 = nn.Linear(512, 256)
        self.hidden4 = nn.Linear(256, 128)
        self.hidden5 = nn.Linear(128, 64)
        self.hidden6 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
    def forward(self, x):
        x = F.leaky_relu(self.input(x))
        x = F.leaky_relu(self.hidden1(x))
        x = F.leaky_relu(self.hidden2(x))
        x = F.leaky_relu(self.hidden3(x))
        x = F.leaky_relu(self.hidden4(x))
        x = F.leaky_relu(self.hidden5(x))
        x = F.leaky_relu(self.hidden6(x))
        x = self.output(x)
        return x

Variables Definintions

In [ ]:
t = int(time.time())
useCUDA = True
epochs = 1000
batchSize = 2048

dir = os.getcwd()
modelFolder = f"{dir}/../trained_models/"
if not os.path.exists(modelFolder):
	os.makedirs(modelFolder)

modelPath = f"{modelFolder}{t}-{epochs}-{batchSize}-{inputSize}.pt"

Device Check

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if not useCUDA:
	device = torch.device('cpu')

Model and Dataset Creation

In [ ]:
net = Net()
net = net.to(device)

In [ ]:
trainSetX = torch.load("../dataset/trainSetX.pt")
trainSetY = torch.load("../dataset/trainSetY.pt")
trainSetX = trainSetX.to(device)
trainSetY = trainSetY.to(device)
print(trainSetX.shape)
print(trainSetY.shape)

In [ ]:
testSetX = torch.load("../dataset/testSetX.pt")
testSetY = torch.load("../dataset/testSetY.pt")
testSetX = testSetX.to(device)
testSetY = testSetY.to(device)
print(testSetX.shape)
print(testSetY.shape)

In [ ]:
trainSet = TensorDataset(trainSetX, trainSetY)
trainLoader = DataLoader(trainSet, batch_size=batchSize, shuffle=True)

In [ ]:
testSet = TensorDataset(testSetX, testSetY)
testLoader = DataLoader(testSet, batch_size=len(testSet), shuffle=True)

Training

In [ ]:
optimizer = optim.Adam(net.parameters(), lr =1e-5)

trainCriterion = nn.L1Loss(reduction='mean')
testCriterion = nn.L1Loss(reduction='mean')

print("Epochs Started")

bestLoss = float('inf')

for epoch in range(epochs):
	net.train()
	running_loss = 0.0
	torch.cuda.empty_cache()
	for i, data in enumerate(trainLoader):
		X, y = data
		y = y.unsqueeze(1)

		net.zero_grad()
		output = net(X)
		loss = trainCriterion(output, y)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()
		# if i % 1000 == 999:    # print every 1000 mini-batches
	print(f'Epoch {epoch+1}, Train loss: {running_loss/len(trainLoader):.3f}')
	running_loss = 0.0
	
	
	torch.cuda.empty_cache()
	net.eval()
	with torch.no_grad():
		for i, data in enumerate(testLoader):
			X, y = data
			y = y.unsqueeze(1)
			output = net(X)
			testLoss = testCriterion(output, y)
			print(f'Epoch {epoch+1}, Test loss: {testLoss.item():.3f}')
			if testLoss.item() < bestLoss:
				bestLoss = testLoss.item()
				torch.save(net.state_dict(), modelPath)
				print("Model Saved")

torch.cuda.empty_cache()